<a href="https://colab.research.google.com/github/gabrielxbox/projeto-com-python/blob/master/Vetoriza%C3%A7%C3%A3o_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import svd
from nltk.stem import RSLPStemmer
import multiprocessing as mp
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(“Tweets_Mg.csv”)
df.head(10)

df = df.Text
df.head(10)

O primeiro passo é separar as palavras. Vamos começar montando um dicionário
com todas elas e quantas vezes aparecem em cada documento, ou seja, vamos contar.
Começamos pela função que conta as palavras de um texto.

In [ ]:
def conta_palavras(texto):
 palavras = texto.split(“ “)
 return {x:palavras.count(x) for x in palavras}

In [ ]:
conta_palavras(df.values[0])

Antes de apenas contar, precisamos executar alguns passos para eliminar coisas como
estas: palavras de pouco significado semântico (como as preposições de, para, em
etc.), padronizar as palavras de forma que o tempo verbal, plural ou singular, gênero e
outros diferenciais não interfiram.
Vamos remover as preposições, as chamadas stopwords, e depois faremos uso da
classe RSLPStemmer do pacote nltk importado anteriormente. Essa classe executa um
processo chamado stemming. Vejamos a seguir o comportamento desse processo

In [ ]:
stemer = RSLPStemmer()

In [ ]:
stemer.stem(“Governador”)

In [ ]:
stemer.stem(“Governadora”)

In [ ]:
stemer.stem(“Governadoras”)

In [ ]:
stemer.stem(“Valadares/MG”)

In [ ]:
stemer.stem(“https://t.co/JSbKamIqUJ”)

In [ ]:
stemer.stem(“���⛪”)

In [ ]:
stemer.stem(“@”)

Perceba que as expressões foram normalizadas para caixa baixa, e tiveram o número
(plural ou singular) e o gênero removidos.
Quanto às palavras de pouco significado, elas são as chamadas stopwords. Vamos
remover essas palavras do texto. Vamos começar carregando essas palavras.

In [ ]:
stopwords = pd.read_csv(“stopwords.csv”)
stopwords.head(10)

In [ ]:
stopwords = stopwords.values
stopwords[:10]

In [ ]:
stopwords = [linha[0].strip() for linha in stopwords]
stopwords

In [ ]:
def conta_palavras(texto, stopwords=stopwords):
 palavras = texto.split(“ “)
 palavras = [palavra for palavra in palavras if palavra not
in stopwords]
 return {x:palavras.count(x) for x in palavras}

In [ ]:
conta_palavras(df.values[0])

In [ ]:
def conta_palavras(texto, stopwords=stopwords):
 palavras = texto.split(“ “)
 palavras = [palavra for palavra in palavras if palavra not
in stopwords and palavra.isalpha()]
 return {x:palavras.count(x) for x in palavras}

In [ ]:
conta_palavras(df.values[0])

In [ ]:
def conta_palavras(texto, stopwords=stopwords, stemer=stemer):

In [ ]:
palavras = texto.split(“ “)
 palavras = [stemer.stem(palavra) for palavra in palavras if
palavra not in stopwords and palavra.isalpha()]
 return {x:palavras.count(x) for x in palavras}

In [ ]:
conta_palavras(df.values[0])

Agora temos o resultado que queremos. Vamos aplicar isso aos demais textos, vamos usar
a função map do pacote multiprocessing e gerar o resultado em paralelo para otimizar.

In [ ]:
with mp.Pool() as p:
 contagem = p.map(conta_palavras, df.values)

In [ ]:
termo_documento_transposto = pd.DataFrame(contagem)
termo_documento_transposto.head(10)

In [ ]:
termo_documento_transposto = termo_documento_transposto.fillna(0)
termo_documento_transposto.head(100)

In [ ]:
plt.figure(figsize=(20, 20))

In [ ]:
plt.imshow(termo_documento_transposto, cmap=’gray’)

In [ ]:
termo_documento = termo_documento_transposto.T
termo_documento.head(10)

In [ ]:
U, W, Vt = svd(termo_documento.values, full_matrices=True)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(U, cmap=’gray’)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(Vt, cmap=’gray’)

Vamos começar analisando a similaridade semântica de documentos. Vamos começar
colhendo alguns documentos da lista de tweets original:

In [ ]:
indice_amostra = [12, 53, 8, 8188, 8169, 8198, 9, 13, 14]
amostra = df[indice_amostra]
for indice, documento in amostra.iteritems():
 print(indice, “\n”, documento, “\n”, “-”*50, “\n”)

In [ ]:
V = np.transpose(Vt)


In [ ]:
V_amostra = V[indice_amostra]
V_amostra

In [ ]:
V_amostra2 = V_amostra[:,:2]
V_amostra2

In [ ]:
V_amostra2 = np.asarray([v/np.linalg.norm(v) for v in V_amostra2])

In [ ]:
plt.figure(figsize=(12, 12))
x = V_amostra2[:, 0]
y = V_amostra2[:, 1]
plt.scatter(x, y)
for i, indice in enumerate(indice_amostra):
 plt.annotate(indice, (x[i], y[i]))
 

In [ ]:
df[12]

In [ ]:
df[53]